In [1]:
import numpy as np
import pandas as pd

In [2]:
class video(object):
    def __init__(self, idx, size):
        self.idx = idx 
        self.size = size

In [35]:
class request(object):
    def __init__(self, endpoint, videoid, value):
        self.endpoint = endpoint
        self.video = videoid
        self.value = value
        self.bestCacheserver = (-1,-1)
        
    def findbestcacheserver(self,endpoint_list):
        end = endpoint_list[self.endpoint]
        if len(end.cacheservers) == 0:
            self.bestCacheserver = (-1, end.latency_datacenter)
        else:
            self.bestCacheserver = endpoint_list[self.endpoint].cacheservers[0]
        
    def computescore(self,endpoint_list):
        end = endpoint_list[self.endpoint]
        
        self.score = (end.latency_datacenter - self.bestCacheserver[1])*self.value

In [4]:
class endpoint(object):
    def __init__(self,idx, latency_datacenter):
        self.idx = idx
        self.cacheservers = [] #contains cacheservers ids, latencies
        self.latency_datacenter = latency_datacenter

    def addcacheserver(self, cacheserverid, cacheserverlatency):
        self.cacheservers+=[(cacheserverid,cacheserverlatency)]
        
    def sortcacheservers(self):
        self.cacheservers = sorted(self.cacheservers, key = lambda x: x[1])

    def findbestcache(self, videoid, videos, listcacheserver):
        #tester trouve la meilleure latency possible
        L = self.latency_datacenter
        bestCacheserverId=-1
        for c in self.cacheservers:
            if listcacheserver[c[0]].can_add_video_or_video_already_in(videoid,videos):
                if c[1]<L:
                    L=c[1]
                    bestCacheserverId=c[0]
        return L,bestCacheserverId

    def getcurrentlatency(self, videoid, listcacheservers):
        #calcule la latency actuel pour la videoid
        L=self.latency_datacenter
        cacheserverId=-1
        for c in self.cacheservers:
            if listcacheservers[c[0]].isvideoinside(videoid):
                if c[1]<L:
                    L=c[1]
                    cacheserverId=c[0]
        return L,cacheserverId

In [5]:
class cacheserver(object):
    def __init__(self, idx, capacity):
        self.idx = idx 
        self.capacity = capacity
        self.videosid = [] #contains videos

    def addvideo(self, videoid,videos):
        # retourne true et addla video si y a de la place
        # sinon retourne false
        if self.capacity-videos[videoid].size>=0:
            self.capacity-=videos[videoid].size
            self.videosid.append(videoid)
            return True
        else:
            return False

    def isvideoinside(self,videoid):
        return videoid in self.videosid
    
    def canaddvideo(self,videoid,videos):
        return self.capacity-videos[videoid].size>=0

    def can_add_video_or_video_already_in(self,videoid,videos):
        b=(videoid in self.videosid)
        if b==True:
            return b
        else:
            return self.capacity-videos[videoid].size>=0

In [6]:
def create_cacheservers(C,X):
    cserver_list = []
    for i in range(C):
        cserver = cacheserver(i,X)
        cserver_list += [cserver]
    return cserver_list

def create_videos(V,video_sizes):
    vid_list = []
    for i in range(V):
        vid = video(i,video_sizes[i])
        vid_list += [vid]
    return vid_list

def create_requests(R,RV_list,RE_list,RN_list):
    req_list = []
    for i in range(R):
        req = request(RE_list[i],RV_list[i],RN_list[i])
        req_list += [req]
    return req_list

def create_endpoints(E,LD_list,caches_list):
    end_list = []
    for i in range(E):
        end = endpoint(i,LD_list[i])
        for (c,Lc) in caches_list[i]:
            end.addcacheserver(c,Lc)
        end.sortcacheservers()
        end_list += [end]
    return end_list

def read_data(title):
    # reads title
    # returns lists
    # V,E,R,C,X
    # LD_list[i] is the LD of user i
    # K_list[i] is the K of user i
    # caches_list[i] is a list of the cacheserver values (id,latency)
    # RV_list[i] is the video id of request i
    # RE_list[i] is the endpoint id of request i
    # RN_list[i] is the number of requests of request i

    file = open(title,'r')

    LD_list = []
    K_list = []
    caches_list = []
    RV_list = []
    RE_list = []
    RN_list = []

    lines = file.readlines()
    i = 0
    V,E,R,C,X = [int(str) for str in lines[i].split()]
    i+= 1
    video_sizes = [int(str) for str in lines[i].split()]
    i += 1

    for e in range(E):
        LD,K = [int(str) for str in lines[i].split()]
        LD_list += [LD]
        K_list += [K]
        i += 1
        caches = []
        for l in range(K):
            c,Lc = [int(str) for str in lines[i].split()]
            caches += [(c,Lc)]
            i += 1
        caches_list += [caches]
    for r in range(R):
        RV,RE,RN = [int(str) for str in lines[i].split()]
        RV_list += [RV]
        RE_list += [RE]
        RN_list += [RN]
        i += 1
        
    return V,E,R,C,X,video_sizes,LD_list,K_list,caches_list,RV_list,RE_list,RN_list

def write_sub(name,N,cacheserver_list):
    file = open(name,'w')
    file.write(str(N)+'\n')
    for i in range(N):
        cacheserver = cacheserver_list[i]
        idx = cacheserver.idx
        vids = cacheserver.videosid
        towrite = str(idx)
        for vid in vids:
            towrite += ' ' + str(vid)
        towrite += '\n'
        file.write(towrite)

In [7]:
name_list = ['kittens','me_at_the_zoo','trending_today','videos_worth_spreading']

In [33]:
def fill_request(req,endpoint_list,video_list,cacheserver_list):
    v = req.video
    vid = video_list[v]
    e = req.endpoint
    end = endpoint_list[e]
    sortedcservers = end.cacheservers
    if len(sortedcservers) > 0:
        for c,L in sortedcservers:
            cserver = cacheserver_list[c]
            if cserver.isvideoinside(v):
                break
            else:
                if cserver.canaddvideo(v,video_list):
                    cserver.addvideo(v,video_list)
                    break

In [9]:
def read_do_write2(name_list):
    for name in name_list:
        V,E,R,C,X,video_sizes,LD_list,K_list,caches_list,RV_list,RE_list,RN_list = read_data(name+'.in')
        cacheserver_list = create_cacheservers(C,X)
        video_list = create_videos(V,video_sizes)
        request_list = create_requests(R,RV_list,RE_list,RN_list)
        endpoint_list = create_endpoints(E,LD_list,caches_list)
        
        for req in request_list:
            fill_request(req,endpoint_list,video_list,cacheserver_list)
                
        write_sub('sub2_'+name+'.txt', len(cacheserver_list) , cacheserver_list)

In [23]:
def sort_requests(request_list,endpoint_list):
    for req in request_list:
        req.findbestcacheserver(endpoint_list)
        req.computescore(endpoint_list)
    zipped = list(zip(request_list,[req.score for req in request_list]))
    sorted_zipped = sorted(zipped, key = lambda x: x[1],reverse=True)
    unz1,unz2 = zip(*sorted_zipped)
    return list(unz1)

In [25]:
def read_do_write3(name_list):
    for name in name_list:
        V,E,R,C,X,video_sizes,LD_list,K_list,caches_list,RV_list,RE_list,RN_list = read_data(name+'.in')
        cacheserver_list = create_cacheservers(C,X)
        video_list = create_videos(V,video_sizes)
        request_list = create_requests(R,RV_list,RE_list,RN_list)
        endpoint_list = create_endpoints(E,LD_list,caches_list)
        
        sorted_request_list = sort_requests(request_list,endpoint_list)
        
        for req in sorted_request_list:
            fill_request(req,endpoint_list,video_list,cacheserver_list)
                
        write_sub('sub3_'+name+'.txt', len(cacheserver_list) , cacheserver_list)

In [36]:
read_do_write3(['videos_worth_spreading'])

In [27]:
V,E,R,C,X,video_sizes,LD_list,K_list,caches_list,RV_list,RE_list,RN_list = read_data('videos_worth_spreading.in')
cacheserver_list = create_cacheservers(C,X)
video_list = create_videos(V,video_sizes)
request_list = create_requests(R,RV_list,RE_list,RN_list)
endpoint_list = create_endpoints(E,LD_list,caches_list)

In [28]:
len(endpoint_list)

100

In [30]:
max([req.endpoint for req in request_list])

99

In [24]:
[(req.bestCacheserver,req.score) for req in sort_requests(request_list,endpoint_list)]

[((496, 1), 29957011),
 ((496, 1), 29951013),
 ((89, 1), 29868144),
 ((197, 1), 29864026),
 ((496, 1), 29861043),
 ((496, 1), 29846048),
 ((42, 1), 29836112),
 ((42, 1), 29827154),
 ((42, 1), 29779378),
 ((42, 1), 29758476),
 ((197, 1), 29750520),
 ((197, 1), 29741559),
 ((105, 3), 29732494),
 ((117, 1), 29698188),
 ((42, 1), 29668896),
 ((197, 1), 29654936),
 ((496, 1), 29642116),
 ((496, 1), 29642116),
 ((344, 5), 29635910),
 ((156, 1), 29621924),
 ((496, 1), 29615125),
 ((89, 1), 29544792),
 ((117, 1), 29533298),
 ((496, 1), 29474172),
 ((247, 1), 29440974),
 ((186, 5), 29435616),
 ((247, 1), 29432100),
 ((395, 3), 29411613),
 ((269, 1), 29386094),
 ((42, 1), 29385226),
 ((344, 5), 29380146),
 ((395, 3), 29379042),
 ((269, 1), 29374198),
 ((197, 1), 29365197),
 ((344, 5), 29359328),
 ((255, 2), 29340636),
 ((28, 4), 29335358),
 ((344, 5), 29332562),
 ((496, 1), 29327221),
 ((269, 1), 29320666),
 ((269, 1), 29320666),
 ((496, 1), 29312226),
 ((105, 3), 29306211),
 ((117, 1), 29302452

In [14]:
l = zip([0,1,2],[5,3,6])

In [16]:
l[0]

TypeError: 'zip' object is not subscriptable

In [10]:
endpoint_list[0].cacheservers

[(12, 1),
 (275, 2),
 (37, 3),
 (176, 3),
 (63, 6),
 (52, 8),
 (121, 10),
 (177, 11),
 (162, 13),
 (349, 13),
 (102, 14),
 (178, 14),
 (4, 15),
 (494, 15),
 (267, 20),
 (142, 21),
 (64, 26),
 (104, 26),
 (336, 26),
 (416, 27),
 (388, 33),
 (360, 36),
 (44, 37),
 (116, 38),
 (376, 38),
 (412, 38),
 (13, 39),
 (0, 41),
 (186, 41),
 (17, 43),
 (97, 43),
 (131, 44),
 (51, 46),
 (209, 46),
 (307, 47),
 (448, 47),
 (351, 51),
 (355, 51),
 (409, 53),
 (286, 54),
 (109, 55),
 (191, 58),
 (491, 59),
 (329, 60),
 (369, 60),
 (8, 61),
 (306, 61),
 (493, 61),
 (468, 63),
 (276, 64),
 (334, 64),
 (474, 64),
 (69, 68),
 (375, 70),
 (143, 75),
 (406, 77),
 (430, 79),
 (103, 80),
 (39, 83),
 (105, 83),
 (312, 84),
 (499, 85),
 (381, 87),
 (266, 88),
 (241, 90),
 (128, 91),
 (302, 91),
 (256, 93),
 (402, 93),
 (318, 94),
 (261, 97),
 (278, 98),
 (284, 98),
 (31, 100),
 (495, 100),
 (498, 102),
 (195, 104),
 (236, 104),
 (181, 106),
 (245, 106),
 (208, 107),
 (252, 107),
 (325, 109),
 (393, 109),
 (391,